# Homework 2: Word Similarity

Student Name: Zhouhui Wu

Student ID: 963830

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

(1 mark)

In [28]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###

# the whole process can be divided into 3 part:
# 1. read and store all alphabetic pairs
# 2. compute docuemnt frequency using Brown corpus
# 3. filter word pairs using document frequency information
# follow this approach, we can filter those pairs without storing Brown corpus informaiton and processing
# Brown corpus only 1 time

# lemmatize function from tutorial 1
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


# read and store all alphabetic word pairs
all_pairs={} # used to store all pair information
all_words=set() # used to store all word types
file = open('set1.tab', 'r')
for line in file:
    tokens=line.split('\t')
    word_1=tokens[0]
    word_2=tokens[1]
    if word_1.isalpha() and word_2.isalpha():
        # 2 words are alphabetic
        similarity=float(tokens[2])
        all_pairs[(word_1,word_2)]=similarity
        if word_1 not in all_words:
            all_words.add(word_1)
        if word_2 not in all_words:
            all_words.add(word_2)
file.close()

document_frequency={} # a dictionary used to store document frequency for each word

# initialize the document frequency dictionary
for word in all_words:
    document_frequency[word]=0

# compute document frequency

paragraphs=brown.paras()
for paragraph in paragraphs:
    vocabulary=set() # a set used to store all word types in a paragraph
    for sentence in paragraph:
        for word in sentence:
            if not word.isalpha():
                continue
            word=lemmatize(word.lower())
            if word not in vocabulary:
                vocabulary.add(word)

    for word in vocabulary:
        if word in document_frequency:
            document_frequency[word]+=1

# filter with document frequency        
for pair in all_pairs:
    word_1=pair[0]
    word_2=pair[1]
    word_1_frequency=document_frequency[word_1]
    word_2_frequency=document_frequency[word_2]
    if word_1_frequency>=8 and word_2_frequency>=8:
        # meet document frequency requirement
        filtered_gold_standard[pair]=all_pairs[pair]


###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to /Users/zhouhuiwu/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhouhuiwu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'se

<b>For your testing: </b>

In [29]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [30]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [31]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###

primary_sense={} # used to store primary sense for each word

for pair in filtered_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    word_1_sense,word_2_sense={},{} # dictionary store the synset information and corresponding count
    word_1_gold_standard,word_2_gold_standard=True,True # boolean denoting whether the word should be kept

    # processing word 1
    if len(wordnet.synsets(word_1))!=1:
        # word 1 has more than 1 synsets
        for synset in wordnet.synsets(word_1):
            for lemma in synset.lemmas():
                # find the lemma using its name
                if str(lemma.name()).lower()==word_1:
                    targetLemma=lemma
                    counts=targetLemma.count()
                    word_1_sense[synset]=counts
                    break
                    
        # sort the synset dictionary and check the primary sense requirement
        sorted_sense = sorted(word_1_sense.items(),key = lambda x:x[1],reverse = True)
        if len(word_1_sense)==0:
            # all count is 0, has no primary sense
            word_1_gold_standard=False
        elif len(word_1_sense)>1:
            # more than 1 sense, check the count requirement
            primary_sense[word_1]=sorted_sense[0][0]
            if sorted_sense[0][1]<4*sorted_sense[1][1]:
                word_1_gold_standard=False
            if sorted_sense[0][0].pos()!='n':
                word_1_gold_standard=False
        else:
            # has only 1 sense
            primary_sense[word_1]=sorted_sense[0][0]
            if sorted_sense[0][0].pos()!='n':
                word_1_gold_standard=False
    else:
        # word 1 has only 1 synset, it has a single primary sense
        primary_sense[word_1]=wordnet.synsets(word_1)[0]
        if wordnet.synsets(word_1)[0].pos()!='n':
            # not a noun
            word_1_gold_standard=False
            
    # processing word 2
    if len(wordnet.synsets(word_2))!=1:
        # word 2 has more than 1 synsets
        for synset in wordnet.synsets(word_2):
            for lemma in synset.lemmas():
                # find the lemma using its name
                if str(lemma.name()).lower()==word_2:
                    targetLemma=lemma
                    counts=targetLemma.count()
                    word_2_sense[synset]=counts
                    break
                    
        # sort the synset dictionary and check the primary sense requirement
        sorted_sense = sorted(word_2_sense.items(),key = lambda x:x[1],reverse = True)
        if len(word_2_sense)==0:
            # all count is 0, has no primary sense
            word_2_gold_standard=False
        elif len(word_2_sense)>1:
            # more than 1 sense, check the count requirement
            primary_sense[word_2]=sorted_sense[0][0]
            if sorted_sense[0][1]<4*sorted_sense[1][1]:
                word_2_gold_standard=False
            if sorted_sense[0][0].pos()!='n':
                word_2_gold_standard=False
        else:
            # has only 1 sense
            primary_sense[word_2]=sorted_sense[0][0]
            if sorted_sense[0][0].pos()!='n':
                word_2_gold_standard=False
            
    else:
        # word 2 has only 1 synset, it has a single primary sense
        primary_sense[word_2]=wordnet.synsets(word_2)[0]
        if wordnet.synsets(word_2)[0].pos()!='n':
            # not a noun
            word_2_gold_standard=False
    
    # complete the final dictionary
    if word_1_gold_standard and word_2_gold_standard:
        final_gold_standard[pair]=filtered_gold_standard[pair]

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [32]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [33]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [34]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

for pair in final_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    # get primary sense computed in previous question
    word_1_sense,word_2_sense=primary_sense[word_1],primary_sense[word_2]
    # compute lin similarity
    lin_similarities[pair]=word_1_sense.lin_similarity(word_2_sense,wordnet_ic.ic('ic-brown.dat'))

###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/zhouhuiwu/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psy

<b>For your testing:</b>

In [35]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [36]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###

import math

# similar to the process in question 1, we divide the process into 3 parts:
# 1. find all words and initialize dictionaries used to store document frequency information
# 2. compute docuemnt frequency using Brown corpus
# 3. compute NPMI using document frequency information
# follow this approach, we can compute the NPMI without storing Brown corpus informaiton and processing
# Brown corpus only 1 time

pair_document_frequency={} # used to store document frequency for each pair of words
word_document_frequency={} # used to store document frequency for each word
all_word_types=0 # sum of word types in all paragraphs

# initialize document frequency dictionaries
for pair in final_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    if word_1 not in word_document_frequency:
        word_document_frequency[word_1]=0
    if word_2 not in word_document_frequency:
        word_document_frequency[word_2]=0
    pair_document_frequency[pair]=0

# process the Brown corpus, compute the document frequency
for paragraph in paragraphs:
    words=set() # used to store all word types in a paragraph
    for sentence in paragraph:
        for word in sentence:
            if not word.isalpha():
                continue
            word=lemmatize(word.lower())
            if word not in words:
                words.add(word)
    
    # update the word types sum
    all_word_types+=len(words)
    
    # update document frequency for each word
    for word in words:
        if word in word_document_frequency:
            word_document_frequency[word]+=1
    
    # update document frequency for each pair of words
    for pair in final_gold_standard:
        word_1=pair[0]
        word_2=pair[1]
        if word_1 in words and word_2 in words:
            pair_document_frequency[pair]+=1
                    
# compute the NPMI using document frequency information
for pair in final_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    p_1=word_document_frequency[word_1]/all_word_types
    p_2=word_document_frequency[word_2]/all_word_types
    p_1_2=pair_document_frequency[pair]/all_word_types
    if p_1_2!=0:
        NPMI_similarities[pair]=math.log(p_1*p_2,2)/math.log(p_1_2,2)-1
    else:
        NPMI_similarities[pair]=-1


###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6531272737497535, ('professor', 'doctor'): -1, ('student', 'professor'): 0.535495995400926, ('stock', 'egg'): 0.3685128922213745, ('money', 'cash'): 0.44493834723517267, ('king', 'queen'): 0.4181407297139579, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254173, ('baby', 'mother'): 0.5149353890388502, ('car', 'automobile'): 0.5430334549802613, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.586151062984337, ('brother', 'monk'): 0.4299318525693172, ('journey', 'car'): -1, ('coast', 'hill'): 0.3393102883484622, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892864, ('psychology', 'doctor'): 0.46132815837264096, ('psychology', 'mind'): 0.44616660434351796, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534866, ('planet', 'moon'): 0.6580006162323315, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [37]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [52]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
import numpy as np

# build document-term matrix
all_words = []
for paragraph in paragraphs:
    words={} # used to store all word types in a paragraph
    for sentence in paragraph:
        for word in sentence:
            if not word.isalpha():
                continue
            word=lemmatize(word.lower())
            if word not in words:
                words[word]=1
    all_words.append(words)

vectorizer = DictVectorizer()
matrix = vectorizer.fit_transform(all_words).transpose()

word_types_list = vectorizer.get_feature_names()

# print(matrix.shape)

# approximate the matrix with lower rank matrix
svd = TruncatedSVD(n_components=500)
matrix = svd.fit_transform(matrix)
# print(matrix.shape)

# use dense vector to compute similarity
for pair in final_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    # get dense vectors
    vector_1=matrix[word_types_list.index(word_1)]
    vector_2=matrix[word_types_list.index(word_2)]
    # compute cosine similarity
    norm_1,norm_2,dot_product=0,0,0
    for (i,j) in zip(vector_1,vector_2):
        dot_product+=i*j
        norm_1+=i*i
        norm_2+=j*j
    similarity=dot_product/math.sqrt(norm_1*norm_2)
    LSA_similarities[pair]=similarity

###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.3083339795506384, ('professor', 'doctor'): 0.054570206159738587, ('student', 'professor'): 0.2914452587867365, ('stock', 'egg'): 0.1217006734864723, ('money', 'cash'): 0.14462710983769164, ('king', 'queen'): 0.14137316525678092, ('bishop', 'rabbi'): 0.004680818869042214, ('football', 'basketball'): 0.2747540386019898, ('football', 'tennis'): 0.11254884212829978, ('alcohol', 'chemistry'): 0.11278509563625824, ('baby', 'mother'): 0.36119876348747404, ('car', 'automobile'): 0.343816314635744, ('journey', 'voyage'): 0.06331032143192379, ('coast', 'shore'): 0.4327455296784078, ('brother', 'monk'): 0.0825151570339918, ('journey', 'car'): 0.0027866619150669417, ('coast', 'hill'): 0.22965982336217408, ('forest', 'graveyard'): 0.05510989796371968, ('monk', 'slave'): -0.03700782489579195, ('coast', 'forest'): 0.13865821135162526, ('psychology', 'doctor'): 0.11258559419197485, ('psychology', 'mind'): 0.1137522924774776, ('psychology', 'health'): 0.07552159959228467, ('psyc

<b>For your testing:</b>

In [54]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [42]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

LSA_list=[]
gold_standard_list=[]
lin_list=[]
NPMI_list=[]
for pair in final_gold_standard:
    gold_standard_list.append(final_gold_standard[pair])
    lin_list.append(lin_similarities[pair])
    NPMI_list.append(NPMI_similarities[pair])
    LSA_list.append(LSA_similarities[pair])

pearson_correlations['lin']=pearsonr(lin_list,gold_standard_list)[0]
pearson_correlations['NPMI']=pearsonr(NPMI_list,gold_standard_list)[0]
pearson_correlations['LSA']=pearsonr(LSA_list,gold_standard_list)[0]


###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.18799888869410047, 'LSA': 0.3781659470049462}


<b>For your testing:</b>

In [43]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

In [44]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from scipy import linalg, dot
from nltk.corpus import brown
from gensim.models import KeyedVectors

sentences = brown.sents()

# preprocess the Brown corpus
preprocessed_sentences=[]
for sentence in sentences:
    preprocessed_sentence=[]
    for word in sentence:
        if not word.isalpha():
            continue
        word=lemmatize(word.lower())
        preprocessed_sentence.append(word)
    preprocessed_sentences.append(preprocessed_sentence)
print('finish preprocessing')
    
# train the model
path = get_tmpfile("word2vec.model")
model = Word2Vec(preprocessed_sentences, size=500, iter=50)
model.save("word2vec.model")
vectors = KeyedVectors.load("word2vec.model", mmap='r')
print('finish model training')

# compute similarities
w2v_similarities={}
for pair in final_gold_standard:
    word_1=pair[0]
    word_2=pair[1]
    vector_1 = vectors[word_1]
    vector_2 = vectors[word_2]
    similarity = dot(vector_1,vector_2.T)/(linalg.norm(vector_1)*linalg.norm(vector_2))
    w2v_similarities[pair]=similarity
print(w2v_similarities)

# compute the Pearson correlation co-efficient with the gold standard
w2v_list=[]
new_gold_standard_list=[]
for pair in w2v_similarities:
    new_gold_standard_list.append(final_gold_standard[pair])
    w2v_list.append(w2v_similarities[pair])
pearson_correlations['word2vec']=pearsonr(w2v_list,new_gold_standard_list)[0]
print(pearson_correlations)

finish preprocessing
finish model training
{('bread', 'butter'): 0.60929621195955, ('professor', 'doctor'): 0.13270619547372384, ('student', 'professor'): 0.2827866549264933, ('stock', 'egg'): 0.17539202803577916, ('money', 'cash'): 0.21923536714620365, ('king', 'queen'): 0.18114439638171745, ('bishop', 'rabbi'): 0.25529896186068524, ('football', 'basketball'): 0.4959148806803802, ('football', 'tennis'): 0.527736858440259, ('alcohol', 'chemistry'): 0.1525535423874349, ('baby', 'mother'): 0.28030245483292293, ('car', 'automobile'): 0.2706369588962117, ('journey', 'voyage'): 0.37572879186549063, ('coast', 'shore'): 0.4359815046319264, ('brother', 'monk'): 0.19025000160991914, ('journey', 'car'): 0.11992453695818724, ('coast', 'hill'): 0.4292015507157831, ('forest', 'graveyard'): 0.15013432735323806, ('monk', 'slave'): 0.0901824155165461, ('coast', 'forest'): 0.22530448697982403, ('psychology', 'doctor'): 0.05234123876911611, ('psychology', 'mind'): 0.075524227236294, ('psychology', 'heal

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.